In [309]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# data sources:
# college tuition, diversity, & pay: https://www.kaggle.com/datasets/jessemostipak/college-tuition-diversity-and-pay
# us college data: https://www.kaggle.com/datasets/yashgpt/us-college-data?resource=download

# load csvs into dataframes
diversity_df = pd.read_csv('archive/diversity_school.csv')
historical_df = pd.read_csv('archive/historical_tuition.csv')
salary_df = pd.read_csv('archive/salary_potential.csv')
tuition_cost_df = pd.read_csv('archive/tuition_cost.csv')
tuition_income_df = pd.read_csv('archive/tuition_income.csv')

## Exploratory Data Analysis

### Dataset 1: College Tuition, Diversity, & Pay

In [310]:
# diversity

# rearrange dataframe

school_cat_count_dict = {}
schools = diversity_df['name'].unique().tolist()
cats = diversity_df['category'].unique().tolist()
cats.append('Total Enrollment')

for school in schools:
    try:
        school_df = diversity_df[diversity_df['name']== school]
        counts = school_df['enrollment'].values.tolist()
        counts.append(school_df['total_enrollment'].values[0])
        school_dict = {cat:count for cat,count in zip(cats,counts)}
        school_cat_count_dict[school] = school_dict
    except IndexError:
        pass
    

In [311]:
new_div_df = pd.DataFrame.from_dict(school_cat_count_dict).T.reset_index().rename(columns = {'index':'College Name'})
new_div_df = new_div_df.drop(columns = ['Unknown'])
print(new_div_df.head())

                                 College Name   Women  \
0               University of Phoenix-Arizona  134722   
1  Ivy Tech Community College-Central Indiana   53476   
2                          Liberty University   48329   
3                    Lone Star College system   41268   
4                          Miami Dade College   38323   

   American Indian / Alaska Native  Asian  Black  Hispanic  \
0                              876   1959  31455     13984   
1                              357   1369  12370      5533   
2                              447    856  14751      1186   
3                              168   4198  12094     23751   
4                               47    655  10722     44870   

   Native Hawaiian / Pacific Islander  White  Two Or More Races  \
0                                1019  58209              19039   
1                                   0  61498               1804   
2                                 151  39062               1772   
3               

In [312]:
# salary
salary_df = salary_df.drop(columns = ['rank','state_name','make_world_better_percent']).rename(
columns = {'name':'College Name'})

print(salary_df.head())

                          College Name  early_career_pay  mid_career_pay  \
0                    Auburn University             54400          104500   
1  University of Alabama in Huntsville             57500          103900   
2            The University of Alabama             52300           97400   
3                  Tuskegee University             54500           93500   
4                   Samford University             48400           90500   

   stem_percent  
0            31  
1            45  
2            15  
3            30  
4             3  


In [313]:
# tuition cost
tuition_cost_df = tuition_cost_df.rename(columns = {'name':'College Name', 'state_code':'State abbreviation', 
                                                   'degree_length':'Degree Length', 'room_and_board':'Room and Board',
                                                   'in_state_tuition':'In-state Tuition',
                                                    'in_state_total':'In-state Total',
                                                   'out_of_state_tuition':'Out-of-state Tuition', 
                                                    'out_of_state_total':'Out-of-state Total'})
print(tuition_cost_df.head())

                           College Name       state State abbreviation  \
0                Aaniiih Nakoda College     Montana                 MT   
1          Abilene Christian University       Texas                 TX   
2  Abraham Baldwin Agricultural College     Georgia                 GA   
3                       Academy College   Minnesota                 MN   
4             Academy of Art University  California                 CA   

         type Degree Length  Room and Board  In-state Tuition  In-state Total  \
0      Public        2 Year             NaN              2380            2380   
1     Private        4 Year         10350.0             34850           45200   
2      Public        2 Year          8474.0              4128           12602   
3  For Profit        2 Year             NaN             17661           17661   
4  For Profit        4 Year         16648.0             27810           44458   

   Out-of-state Tuition  Out-of-state Total  
0                  238

In [314]:
# tuition income - scholarship / time evolved pricing (to be used in later parts of the project)
print(tuition_income_df.head())

                                name state  total_price  year     campus  \
0  Piedmont International University    NC        20174  2016  On Campus   
1  Piedmont International University    NC        20174  2016  On Campus   
2  Piedmont International University    NC        20174  2016  On Campus   
3  Piedmont International University    NC        20174  2016  On Campus   
4  Piedmont International University    NC        20514  2017  On Campus   

      net_cost         income_lvl  
0  11475.00000        0 to 30,000  
1  11451.00000   30,001 to 48,000  
2  16229.00000   48_001 to 75,000  
3  15592.00000  75,001 to 110,000  
4  11668.39249        0 to 30,000  


### Dataset 2: US College Data

In [315]:
college_df = pd.read_csv('archive/College_Data.csv').drop(columns = ['Private', 'perc.alumni','Outstate','Room.Board',
                                                                     'Top10perc','Top25perc','Terminal', 'Enroll'])


college_df = college_df.rename(columns = {'Unnamed: 0':'College Name','Expend':'Avg Instructional Expenditure per Student',
                                          'F.Undergrad':'Full-time Undergrads','P.Undergrad':'Part-time Undergrads',
                                         'Grad.Rate':'Graduation Rate', 'S.F.Ratio':'Student:Faculty Ratio',
                                         'PhD':'Percent of Faculty with PhD', 'Books':'Avg Book Costs',
                                         'Personal':'Avg Personal Costs'})
college_df['Percent of Applications Accepted'] = round(college_df['Accept']/college_df['Apps']*100,0)
college_df = college_df.drop(columns = 'Accept')

print(college_df.head())

                   College Name  Apps  Full-time Undergrads  \
0  Abilene Christian University  1660                  2885   
1            Adelphi University  2186                  2683   
2                Adrian College  1428                  1036   
3           Agnes Scott College   417                   510   
4     Alaska Pacific University   193                   249   

   Part-time Undergrads  Avg Book Costs  Avg Personal Costs  \
0                   537             450                2200   
1                  1227             750                1500   
2                    99             400                1165   
3                    63             450                 875   
4                   869             800                1500   

   Percent of Faculty with PhD  Student:Faculty Ratio  \
0                           70                   18.1   
1                           29                   12.2   
2                           53                   12.9   
3             

## Cleaning & Merging Data

In [316]:
merged_df = new_div_df.merge(salary_df.merge(tuition_cost_df.merge(college_df)))
merged_df = merged_df.fillna(0) # make world better, room and board nans ==> 0

In [318]:
merged_df = merged_df.rename(
    columns = {'Women':'Female Students',
               'American Indian / Alaska Native': 'American Indian / Alaska Native Students',
               'Asian':'Asian Students',
               'Black':'Black Students',
               'Hispanic':'Hispanic Students',
               'Native Hawaiian / Pacific Islander': 'Native Hawaiian / Pacific Islander Students',
               'White':'White Students',
               'Two Or More Races':'Multi-racial Students',
               'Non-Resident Foreign':'Foreign Exchange Students',
                'early_career_pay': 'Avg Early Career Pay',
               'mid_career_pay':'Avg Mid Career Pay',
               'stem_percent':'Percent STEM Majors',
               'state':"State",
               'type':'College Type',
               'Apps':'# of Applications',
    
              }
)
col_order = ['College Name','Total Enrollment','State', 'State abbreviation', 'Graduation Rate', 'Percent of Applications Accepted',
             'In-state Tuition', 'In-state Total', 'Out-of-state Tuition', 'Out-of-state Total','Room and Board','Avg Book Costs', 'Avg Personal Costs',
             'Female Students', 'American Indian / Alaska Native Students', 'Asian Students', 'Black Students', 'Hispanic Students', 'Native Hawaiian / Pacific Islander Students', 'White Students', 'Multi-racial Students', 'Foreign Exchange Students', 'Total Minority', 
             '# of Applications', 'Full-time Undergrads', 'Part-time Undergrads' , 'Percent of Faculty with PhD', 'Student:Faculty Ratio', 'Avg Instructional Expenditure per Student',
             'Avg Early Career Pay', 'Avg Mid Career Pay', 'Percent STEM Majors', 'College Type', 'Degree Length',]
merged_df = merged_df[col_order]
print(merged_df.columns.tolist())

['College Name', 'Total Enrollment', 'State', 'State abbreviation', 'Graduation Rate', 'Percent of Applications Accepted', 'In-state Tuition', 'In-state Total', 'Out-of-state Tuition', 'Out-of-state Total', 'Room and Board', 'Avg Book Costs', 'Avg Personal Costs', 'Female Students', 'American Indian / Alaska Native Students', 'Asian Students', 'Black Students', 'Hispanic Students', 'Native Hawaiian / Pacific Islander Students', 'White Students', 'Multi-racial Students', 'Foreign Exchange Students', 'Total Minority', '# of Applications', 'Full-time Undergrads', 'Part-time Undergrads', 'Percent of Faculty with PhD', 'Student:Faculty Ratio', 'Avg Instructional Expenditure per Student', 'Avg Early Career Pay', 'Avg Mid Career Pay', 'Percent STEM Majors', 'College Type', 'Degree Length']


In [319]:
# save merged df as csv
import os
if os.path.exists('data.csv'):
    os.remove('data.csv')
merged_df.to_csv('data.csv')